To do:


1.   Test more hyperparameters to get metrics in 0.9 - HuggingFace Trainer has native support for hyperparameter search using either Optuna, Ray Tune, or Weights & Biases.
2.   Data augmentation - use another LLM to do NER categorization of text. and then add that to the training model.



In [ ]:
!pip install -q transformers datasets seqeval optuna torch tqdm evaluate

# Clone the repo
!git clone https://github.com/NER-AncientLanguages/NERAncientGreekML4AL.git
%cd NERAncientGreekML4AL

# Verify data exists
!ls final_dataset/normal/*.conll

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
Cloning into 'NERAncientGreekML4AL'...
remote: Enumerating objects: 251, done.
remote: Total 251 (delta 0), reused 0 (delta 0), pack-reused 251 (from 3)
Receiving objects: 100% (251/251), 106.47 MiB | 16.59 MiB/s, done.
Resolving deltas: 100% (98/98), done.
Updating files: 100% (199/199), done.
Filtering content: 100% (5/5), 522.92 MiB | 144.56 MiB/s, done.
/content/NERAncientGreekML4AL
final_dataset/normal/test.conll   final_dataset/normal/val.conll
final_dataset/normal/train.conll


In [ ]:
import os, warnings, unicodedata, numpy as np
from pathlib import Path
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification,
    TrainingArguments, Trainer, DataCollatorForTokenClassification
)

def read_conll(p: Path):
    """
    Parse CoNLL with format:
        [line_id]  token  [POS]  NER
    Example:
        110089790	βίβλος	O
    Returns: {"tokens": [...], "ner_tags": [...]}
    """
    sents, labs = [], []
    with p.open(encoding="utf-8") as f:
        sent, lab = [], []
        for i, raw in enumerate(f, 1):
            line = raw.strip()
            if not line or line.startswith("#"):
                if sent:
                    sents.append(sent)
                    labs.append(lab)
                    sent, lab = [], []
                continue

            # Split on whitespace (handles tabs and spaces)
            parts = line.split()
            if len(parts) < 2:
                print(f"Warning: Line {i} in {p.name} has <2 columns → SKIPPED")
                print(f"    → {line!r}")
                continue

            if len(parts) == 2:
                token = parts[0]
                ner   = parts[1]
            else:
                token = parts[1]   # skip ID
                ner   = parts[-1]  # last column is NER

            sent.append(unicodedata.normalize("NFC", token))
            lab.append(ner)

        if sent:
            sents.append(sent)
            labs.append(lab)

    print(f"Loaded {len(sents)} sentences from {p.name}")
    return {"tokens": sents, "ner_tags": labs}

# load data
train_path = Path("final_dataset/normal/train.conll")
val_path   = Path("final_dataset/normal/val.conll")
test_path  = Path("final_dataset/normal/test.conll")

raw = {
    "train": read_conll(train_path),
    "validation": read_conll(val_path),
    "test": read_conll(test_path),
}
data = DatasetDict({k: Dataset.from_dict(v) for k, v in raw.items()})

#Model name -------------------------------------------------------------
model_name = "Marijke/AG_BERT_hypopt_NER"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
#------------------------------------------------------------------------

all_labels = sorted({l for s in data["train"]["ner_tags"] for l in s})
label2id   = {l: i for i, l in enumerate(all_labels)}
id2label   = {i: l for l, i in label2id.items()}

#tokenise + align labels
def tokenise_align(example):
    tok = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    aligned = []
    for i, labs in enumerate(example["ner_tags"]):
        word_ids = tok.word_ids(batch_index=i)
        prev = None
        ids  = []
        for wid in word_ids:
            if wid is None:
                ids.append(-100)
            elif wid != prev:
                ids.append(label2id[labs[wid]])
            else:
                ids.append(-100)               # sub-word → ignore
            prev = wid
        aligned.append(ids)
    tok["labels"] = aligned
    return tok

tokenised = data.map(tokenise_align, batched=True,
                     remove_columns=data["train"].column_names)


model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(all_labels),
    id2label=id2label,
    label2id=label2id,
)

collator = DataCollatorForTokenClassification(tokenizer)


Loaded 30686 sentences from train.conll
Loaded 4434 sentences from val.conll
Loaded 4701 sentences from test.conll


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/30686 [00:00<?, ? examples/s]

Map:   0%|          | 0/4434 [00:00<?, ? examples/s]

Map:   0%|          | 0/4701 [00:00<?, ? examples/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/449M [00:00<?, ?B/s]

In [ ]:
import evaluate
from seqeval.metrics import classification_report

def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=2)

    true_labels = []
    pred_labels = []

    for prediction, label in zip(preds, labels):
        true_seq = [id2label[l] for l in label if l != -100]
        pred_seq = [id2label[pred] for pred, l in zip(prediction, label) if l != -100]
        if true_seq:  # Only add if not empty
            true_labels.append(true_seq)
            pred_labels.append(pred_seq)

    if not true_labels:
        return {"precision": 0.0, "recall": 0.0, "f1": 0.0}


    metric = evaluate.load("seqeval")
    results = metric.compute(predictions=pred_labels, references=true_labels)
    print(results)

    return {
      "precision": results["overall_precision"],
      "recall": results['overall_recall'],
      "f1": results["overall_f1"]
    }



In [ ]:
# ------------------------------------------------------------
# Hyper-parameters
# ------------------------------------------------------------
LEARNING_RATE = 3e-5
BATCH_SIZE    = 16
EPOCHS        = 5
WEIGHT_DECAY  = 0.01
WARMUP_RATIO  = 0.06
SEED          = 123
OUTPUT_DIR    = "./tuned_ner_model"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=SEED,
    logging_steps=10,
    save_total_limit=2,
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenised["train"],
    eval_dataset=tokenised["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

#Train the model
print("\nSTARTING TRAINING ...\n")
trainer.train()

#Save the model
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"\nModel saved to {OUTPUT_DIR}")



/tmp/ipython-input-2236131195.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



STARTING TRAINING ...



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.039400,0.099955,0.823198,0.829596,0.826385
2,0.014500,0.113058,0.823062,0.836472,0.829713
3,0.009200,0.123023,0.828902,0.834230,0.831558
4,0.010500,0.140278,0.836154,0.834529,0.835341
5,0.002700,0.146498,0.831911,0.844843,0.838327


{'GRP': {'precision': np.float64(0.7902097902097902), 'recall': np.float64(0.8752904725019365), 'f1': np.float64(0.8305769937522969), 'number': np.int64(1291)}, 'LOC': {'precision': np.float64(0.7377543330821401), 'recall': np.float64(0.7278810408921933), 'f1': np.float64(0.7327844311377245), 'number': np.int64(1345)}, 'PERS': {'precision': np.float64(0.8634880803011292), 'recall': np.float64(0.8487913172175628), 'f1': np.float64(0.8560766264460753), 'number': np.int64(4054)}, 'overall_precision': np.float64(0.8231978641352714), 'overall_recall': np.float64(0.8295964125560538), 'overall_f1': np.float64(0.826384752829065), 'overall_accuracy': 0.9769274815588265}
{'GRP': {'precision': np.float64(0.7739307535641547), 'recall': np.float64(0.8830364058869093), 'f1': np.float64(0.8248914616497829), 'number': np.int64(1291)}, 'LOC': {'precision': np.float64(0.7263389581804842), 'recall': np.float64(0.7360594795539034), 'f1': np.float64(0.7311669128508124), 'number': np.int64(1345)}, 'PERS': {

In [ ]:
#Quick test
from transformers import pipeline
import unicodedata

ner = pipeline("ner", model=OUTPUT_DIR, tokenizer=OUTPUT_DIR,
               aggregation_strategy="simple")

txt = unicodedata.normalize("NFC", """
  ᾿Ανέστη δὲ βασιλεὺς ἕτερος ἐπ᾿ Αἴγυπτον, ὃς οὐκ ᾔδει τὸν ᾿Ιωσήφ.
  εἶπε δὲ τῷ ἔθνει αὐτοῦ· ἰδοὺ τὸ γένος τῶν υἱῶν ᾿Ισραὴλ μέγα πλῆθος καὶ ἰσχύει ὑπὲρ ἡμᾶς·
  δεῦτε οὖν κατασοφισώμεθα αὐτούς, μή ποτε πληθυνθῇ, καὶ ἡνίκα ἂν συμβῇ ἡμῖν πόλεμος,
  προστεθήσονται καὶ οὗτοι πρὸς τοὺς ὑπεναντίους καὶ ἐκπολεμήσαντες ἡμᾶς ἐξελεύσονται ἐκ τῆς γῆς.
  καὶ ἐπέστησεν αὐτοῖς ἐπιστάτας τῶν ἔργων, ἵνα κακώσωσιν αὐτοὺς ἐν τοῖς ἔργοις· καὶ Ισραήλᾠκοδόμησαν πόλεις ὀχυρὰς τῷ Φαραώ, τήν τε Πειθὼ καὶ Ῥαμεσσῆ καὶ ῎Ων, ἥ ἐστιν ῾Ηλιούπολις.
  καθότι δὲ αὐτοὺς ἐταπείνουν, τοσούτῳ πλείους ἐγίγνοντο, καὶ ἴσχυον σφόδρα σφόδρα· καὶ ἐβδελύσσοντο οἱ Αἰγύπτιοι ἀπὸ τῶν υἱῶν ᾿.
  καὶ κατεδυνάστευον οἱ Αἰγύπτιοι τοὺς υἱοὺς ᾿Ισραὴλ βίᾳ καὶ κατωδύνων αὐτῶν τὴν ζωὴν ἐν τοῖς ἔργοις τοῖς σκληροῖς, τῷ πηλῷ καὶ τῇ πλινθείᾳ καὶ πᾶσι τοῖς ἔργοις τοῖς ἐν τοῖς πεδίοις, κατὰ πάντα τὰ ἔργα, ὧν κατεδουλοῦντο αὐτοὺς μετὰ βίας.
""")

merged_results = []

for r in ner(txt):
    if r['word'].startswith("##"):
        merged_results[-1]['word'] += r['word'][2:]  # remove ## and join the subwords together instead of splitting it
        merged_results[-1]['score'] = max(merged_results[-1]['score'], r['score'])
    else:
        merged_results.append(r)

for r in merged_results:
    print(f"{r['word']:<20} → {r['entity_group']:<6} ({r['score']:.3f})")


Device set to use cuda:0


αιγυπτον             → LOC    (0.997)
φαραω                → PERS   (0.999)
ραμεσση              → LOC    (0.960)
αιγυπτιοι            → GRP    (1.000)
αιγυπτιοι            → GRP    (1.000)


In [ ]:
# ------------------------------------------------------------
# Hyper-parameters
# ------------------------------------------------------------
LEARNING_RATE = 3e-5
BATCH_SIZE    = 32
EPOCHS        = 5
WEIGHT_DECAY  = 0.01
WARMUP_RATIO  = 1.0
SEED          = 123
OUTPUT_DIR    = f"./tuned_ner_model_lr{LEARNING_RATE}_bs{BATCH_SIZE}_ep{EPOCHS}"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=SEED,
    logging_steps=10,
    save_total_limit=2,
    report_to=[],
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenised["train"],
    eval_dataset=tokenised["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-2947403601.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.003200,0.153893,0.831669,0.846338,0.838939
2,0.004300,0.156487,0.825432,0.843199,0.834221
3,0.008700,0.161607,0.836168,0.837668,0.836918
4,0.009400,0.156468,0.830458,0.834679,0.832563
5,0.002600,0.167400,0.809701,0.845889,0.827400


{'GRP': {'precision': np.float64(0.7855684138869979), 'recall': np.float64(0.8938807126258714), 'f1': np.float64(0.836231884057971), 'number': np.int64(1291)}, 'LOC': {'precision': np.float64(0.7770545158665582), 'recall': np.float64(0.7100371747211895), 'f1': np.float64(0.742035742035742), 'number': np.int64(1345)}, 'PERS': {'precision': np.float64(0.8644768856447689), 'recall': np.float64(0.8764183522446966), 'f1': np.float64(0.8704066634002938), 'number': np.int64(4054)}, 'overall_precision': np.float64(0.831668625146886), 'overall_recall': np.float64(0.8463378176382661), 'overall_f1': np.float64(0.8389391020891984), 'overall_accuracy': 0.9781849618826277}
{'GRP': {'precision': np.float64(0.7809330628803245), 'recall': np.float64(0.8946553059643687), 'f1': np.float64(0.8339350180505415), 'number': np.int64(1291)}, 'LOC': {'precision': np.float64(0.727866473149492), 'recall': np.float64(0.745724907063197), 'f1': np.float64(0.7366874770473741), 'number': np.int64(1345)}, 'PERS': {'pre

TrainOutput(global_step=4795, training_loss=0.00430331057067708, metrics={'train_runtime': 1146.6349, 'train_samples_per_second': 133.809, 'train_steps_per_second': 4.182, 'total_flos': 7583996353035300.0, 'train_loss': 0.00430331057067708, 'epoch': 5.0})

Zero-shot transfer:  Due to its nature that the ground
truth label exists at word level, supervised training of NER models often requires large amount
of human annotation efforts. In real-world use
cases where one needs to build multi-lingual models, the required human labor scales at least linearly with number of languages, or even worse
for low resource languages. Li, B., He, Y., & Xu, W. (2021). Cross-Lingual Named Entity Recognition Using Parallel Corpus: A New Approach Using XLM-RoBERTa Alignment. arXiv
https://arxiv.org/pdf/2101.11112



In [ ]:
# https://huggingface.co/datasets/hmcgovern/original-language-bibles-greek
import pandas as pd

dataset_hf_path = '/content/drive/MyDrive/Deep Learning Group Project/train-00000-of-00001.parquet'
dataset_hf = pd.read_parquet(dataset_hf_path)
dataset_hf.head(10)
len(dataset_hf)

142097

In [ ]:
import os
import torch
import joblib
import pandas as pd
from tqdm.auto import tqdm
from transformers import pipeline
from datasets import Dataset

# ----- SETTINGS -----
CACHE_FILE     = "/content/drive/MyDrive/Deep Learning Group Project/ner_labels_wordlevel.pkl.gz"
CSV_OUT_FILE   = "/content/drive/MyDrive/Deep Learning Group Project/ner_labels.csv"
WORDS_PER_BATCH = 64          # how many *words* to join into one fake sentence since the dataset is on word
HF_BATCH_SIZE   = 64          # pipeline internal batch size (sentences)
# -------------------------

device = 0 if torch.cuda.is_available() else -1
print(f"NER device: {'GPU' if device == 0 else 'CPU'}")

ner_model = pipeline(
    "ner",
    model="dslim/bert-base-NER",
    aggregation_strategy="simple",
    device=device,
    batch_size=HF_BATCH_SIZE,
)

def ner_on_word_batch(word_list):
    if not word_list:
        return []

    fake_sentence = " ".join(word_list)
    entities = ner_model(fake_sentence)
    labels = ["O"] * len(word_list)

    char_pos = 0
    for ent in entities:
        ent_start = ent["start"]
        ent_label = ent["entity_group"]

        word_idx = 0
        current_pos = 0
        while word_idx < len(word_list):
            word_len = len(word_list[word_idx])
            if current_pos <= ent_start < current_pos + word_len:
                labels[word_idx] = ent_label
                break
            current_pos += word_len + 1
            word_idx += 1

    return labels

if os.path.exists(CACHE_FILE):
    print(f"Loading cached NER labels from:\n    {CACHE_FILE}")
    final_labels = joblib.load(CACHE_FILE)
else:
    print(f"Cache missing → running batched NER on {len(dataset_hf)} words...")
    final_labels = []

    words = dataset_hf["translation"].tolist()

    for i in tqdm(range(0, len(words), WORDS_PER_BATCH),
                  desc="NER word batches",
                  total=(len(words) + WORDS_PER_BATCH - 1) // WORDS_PER_BATCH):

        batch_words = words[i:i + WORDS_PER_BATCH]
        batch_labels = ner_on_word_batch(batch_words)
        final_labels.extend(batch_labels)


    os.makedirs(os.path.dirname(CACHE_FILE), exist_ok=True)
    print(f"Saving {len(final_labels)} labels → {CACHE_FILE}")
    joblib.dump(final_labels, CACHE_FILE, compress=("gzip", 3))

dataset_hf = dataset_hf.assign(ner_label=final_labels)

os.makedirs(os.path.dirname(CSV_OUT_FILE), exist_ok=True)
dataset_hf[["text", "translation", "ner_label"]].to_csv(CSV_OUT_FILE, index=False)
print(f"CSV saved: {CSV_OUT_FILE}")


NER device: GPU


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


Loading cached NER labels from:
    /content/drive/MyDrive/Deep Learning Group Project/ner_labels_wordlevel.pkl.gz
CSV saved: /content/drive/MyDrive/Deep Learning Group Project/ner_labels.csv


In [ ]:
### 1. Need to convert the ner label to BIO format first
### 2. Join the new dataset to the existing trainingdata


from datasets import concatenate_datasets
import os

print(f"Original train rows: {len(data['train'])}")
print(f"Original train features: {data['train'].features}")

from datasets import Features, Value, Sequence

new_dataset = Dataset.from_dict({
    "tokens": dataset_hf["text"].tolist(),
    "ner_tags": dataset_hf["ner_label"].tolist(),
})
print(new_dataset)

expected_features = Features({
    "tokens": Sequence(Value("string")),
    "ner_tags": Value("string"),
})

print(f"Adding {len(new_dataset)} NER examples to train...")
data["train"] = concatenate_datasets([data["train"], new_dataset])

print(f"New train size: {len(data['train'])} rows")

print("Tokenizing full train split...")
tokenised_train = data["train"].map(
    tokenise_align,
    batched=True,
    num_proc=os.cpu_count(),
    batch_size=2000,
    remove_columns=data["train"].column_names,  # keep only tokenizer outputs
)

data["train"] = tokenised_train


print(f"Tokenization complete!")
print(f"Final train rows: {len(data['train'])}")
print(f"Final columns: {data['train'].column_names}")
print(f"Example: {data['train'][0].keys()}")

Original train rows: 30686
Original train features: {'tokens': List(Value('string')), 'ner_tags': List(Value('string'))}
Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 142097
})
Adding 142097 NER examples to train...


ValueError: The features can't be aligned because the key tokens of features {'tokens': Value('string'), 'ner_tags': Value('string')} has unexpected type - Value('string') (expected either List(Value('string')) or Value("null").

In [ ]:
#and then save it for future training
def save_hf_dataset_to_conll(dataset, output_file):

    with open(output_file, "w", encoding="utf-8") as f:
        for example in dataset:
            tokens = example["tokens"]
            labels = example["ner_tags"]

            for token, label in zip(tokens, labels):
                f.write(f"{token}\t{label}\n")  # tab-separated
            f.write("\n")  # sentence boundary

# Example: convert train split
save_hf_dataset_to_conll(dataset_hf["train"], "train_from_hf.conll")
save_hf_dataset_to_conll(dataset_hf["test"], "test_from_hf.conll")



In [ ]:
# ------------------------------------------------------------
# Hyper-parameters
# ------------------------------------------------------------
LEARNING_RATE = 3e-5
BATCH_SIZE    = 32
EPOCHS        = 10
WEIGHT_DECAY  = 0.001
WARMUP_RATIO  = 0.6
SEED          = 123
OUTPUT_DIR    = f"./tuned_ner_model_lr{LEARNING_RATE}_bs{BATCH_SIZE}_ep{EPOCHS}"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=SEED,
    logging_steps=10,
    save_total_limit=2,
    report_to=[],
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenised["train"],
    eval_dataset=tokenised["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

trainer.train()